In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d prithwirajmitra/covid-face-mask-detection-dataset
!unzip covid-face-mask-detection-dataset.zip
print("Done")

In [ ]:
import os
import random
import time
import platform
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing, metrics, model_selection
from skimage import io
from PIL import Image
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models, datasets, utils
from torchvision.datasets import ImageFolder

In [ ]:
SEED = 42

random.seed(SEED)
np.random.seed(SEED)

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {DEVICE}')

In [ ]:
data_transforms_train = transforms.Compose([
                                            transforms.RandomResizedCrop(224),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                          ])

data_transforms_test = transforms.Compose([
                                            transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                         ])


In [ ]:
data_dir = '/content/New Masks Dataset'
train_dir = os.path.join(data_dir, 'Train')
val_dir = os.path.join(data_dir, 'Validation')
test_dir = os.path.join(data_dir, 'Test')

train_dataset = ImageFolder(train_dir, transform=data_transforms_train)
val_dataset = ImageFolder(val_dir, transform=data_transforms_test)
test_dataset = ImageFolder(test_dir, transform=data_transforms_test)

BATCH_SIZE = 32
NUM_WORKERS = 2

train_dataloader = DataLoader(
                              train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle = True,
                              num_workers=NUM_WORKERS )

val_dataloader = DataLoader(
                            val_dataset,
                            batch_size=BATCH_SIZE,
                            shuffle = False,
                            num_workers=NUM_WORKERS
                           )

test_dataloader = DataLoader(
                            test_dataset,
                            batch_size=BATCH_SIZE,
                            shuffle = False,
                            num_workers=NUM_WORKERS
                           )

print("DataLoaders criados com sucesso usando ImageFolder.")
print(f"Train batches: {len(train_dataloader)}")
print(f"Validation batches: {len(val_dataloader)}")
print(f"Test batches: {len(test_dataloader)}")

Definição e Ajuste dos Modelos


In [ ]:
from torchvision import models
import torch.nn as nn

num_classes = len(train_dataset.classes)

vgg16 = models.vgg16(pretrained=True)
vgg16.classifier[6] = nn.Linear(4096, num_classes)
vgg16 = vgg16.to(DEVICE)

alexnet = models.alexnet(pretrained=True)
alexnet.classifier[6] = nn.Linear(4096, num_classes)
alexnet = alexnet.to(DEVICE)


Função de Treino e Validação do Dataset


In [ ]:
import torch
from torch import optim
import matplotlib.pyplot as plt

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50, model_name="Modelo"):
    train_loss_list, val_loss_list = [], []
    train_acc_list, val_acc_list = [], []

    for epoch in range(num_epochs):
        print(f"\nÉpoca {epoch+1}/{num_epochs}")
        model.train()
        train_loss, train_corrects = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            train_corrects += torch.sum(preds == labels.data)

        epoch_loss = train_loss / len(train_loader.dataset)
        epoch_acc = train_corrects.double() / len(train_loader.dataset)
        train_loss_list.append(epoch_loss)
        train_acc_list.append(epoch_acc.item())


        model.eval()
        val_loss, val_corrects = 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)

                val_loss += loss.item() * inputs.size(0)
                val_corrects += torch.sum(preds == labels.data)

        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corrects.double() / len(val_loader.dataset)
        val_loss_list.append(val_loss)
        val_acc_list.append(val_acc.item())

        print(f"Treino - Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}")
        print(f"Validação - Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")


    epochs_list = list(range(num_epochs))
    plt.figure(figsize=(8, 5))
    plt.plot(epochs_list, train_loss_list, label="Train Loss", color='magenta')
    plt.plot(epochs_list, val_loss_list, label="Val Loss", color='green')
    plt.title(f"Loss por Época - {model_name}")
    plt.xlabel("Época")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()

    plt.figure(figsize=(8, 5))
    plt.plot(epochs_list, train_acc_list, label="Train Accuracy", color='magenta')
    plt.plot(epochs_list, val_acc_list, label="Val Accuracy", color='green')
    plt.title(f"Acurácia por Época - {model_name}")
    plt.xlabel("Época")
    plt.ylabel("Acurácia")
    plt.legend()
    plt.show()

    return model


In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer_vgg = optim.Adam(vgg16.parameters(), lr=0.0001)
vgg16 = train_model(vgg16, train_dataloader, val_dataloader, criterion, optimizer_vgg,
                    num_epochs=50, model_name="VGG16")

optimizer_alex = optim.Adam(alexnet.parameters(), lr=0.0001)
alexnet = train_model(alexnet, train_dataloader, val_dataloader, criterion, optimizer_alex,
                      num_epochs=50, model_name="AlexNet")


Avaliação com Matriz de Confusão e as Métricas

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def evaluate_model(model, test_loader, model_name):
    model.eval()
    y_true, y_pred = [], []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())


    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5,4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
                xticklabels=train_dataset.classes,
                yticklabels=train_dataset.classes)
    plt.title(f"Matriz de Confusão - {model_name}")
    plt.xlabel("Predito")
    plt.ylabel("Real")
    plt.show()

    print(f"\n--- {model_name} ---")
    print(classification_report(y_true, y_pred, target_names=train_dataset.classes))


In [ ]:
evaluate_model(vgg16, test_dataloader, "VGG16")
evaluate_model(alexnet, test_dataloader, "AlexNet")
